In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import wandb

from utils.neural_network import NeuralNetwork
from utils.wandb_classes import WandbTrainer
from utils.helper_functions import get_optimizer, load_data

In [10]:
wandb.login()

True

In [11]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data('fashion_mnist')

### Trials

In [12]:
nn = NeuralNetwork(layer_sizes=[784, 128, 128, 128, 10], 
                   activation_functions=['relu', 'relu', 'relu', 'softmax'], 
                   weight_init='xavier',
                   weight_decay=0.0, LOG_EACH=True)

H, A = nn.forward_propagation(X_train)
loss = nn.compute_loss(H[-1], y_train)
print(f'{nn.compute_accuracy(X_val, y_val) :>.6f}')

nn.set_optimizer({'name':'nadam', 'learning_rate':0.001, 'epsilon':1e-7, 'beta1':0.95})

0.125000


In [13]:
num_trial_datapoints = 54000

LOG_EACH = True

nn.train(X_train[:num_trial_datapoints], 
         y_train[:num_trial_datapoints], 
         X_val, y_val, 
         batch_size=64, 
         num_epochs=5, 
         loss_type='cross_entropy', 
         log_every=5000)

LOG_EACH = False

print('--'*20,'DONE','--'*20)
print(nn.compute_accuracy(X_test, y_test))

Running NadamOptimizer self.learning_rate = 0.001 self.beta1 = 0.95 self.beta2 = 0.999 self.epsilon = 1e-07
Epoch 1/5, Iteration   0/844 --> Train Loss: 2.26874, Val Loss: 2.17015
---------------------------------------- DONE ----------------------------------------
0.8787


### WANDB SWEEP

In [14]:
# # Create New sweep
# import yaml

# with open("sweep_config.yaml", "r") as file:
#         sweep_config = yaml.safe_load(file)

# sweep_id = wandb.sweep(sweep_config, 
#                        entity="bullseye2608-indian-institute-of-technology-madras",
#                        project="fashion_mnist_hp_search")

# # Run the sweep
# wandb.agent(sweep_id, wandb_sweep_helper_function, count=60)

In [ ]:
# Continue the sweep

sweep_id_cont = "bullseye2608-indian-institute-of-technology-madras/fashion_mnist_hp_search/vhbqpquu"
trainer = WandbTrainer()

wandb.agent(sweep_id_cont, trainer.train, count=100)